# Cek Apakah passage masih memiliki \n\n
\n\n adalah separator antara passage, yg mana jika tidak ada berarti model llm hanya menerima input teks berupa passage pertama saja, dan bukan passage kedua & ketika

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer
from tqdm import tqdm

# Load dataset dan tokenizer
dataset = load_from_disk("./generated_data/RECOMP_tuning_with_no_Judul_nTeks")
test_data = dataset["test"]
tokenizer = AutoTokenizer.from_pretrained("./models/-google-flan-t5-base-2025-05-28_07-17-31")

# Inisialisasi
decoded_passages = []
pipe_counts = []
multi_pipe_count = 0
single_or_none_pipe_count = 0

# Proses setiap passage
for passage in tqdm(test_data["passages"]):
    # Tokenisasi dan truncation
    tokenized = tokenizer(
        passage,
        max_length=512,
        truncation=True,
        return_tensors="pt",
        add_special_tokens=True
    )
    
    # Decode
    decoded = tokenizer.decode(tokenized["input_ids"][0], skip_special_tokens=True)
    decoded_passages.append(decoded)

    # Hitung jumlah karakter pipe
    pipe_count = decoded.count("|")
    pipe_counts.append(pipe_count)

    # Klasifikasi
    if pipe_count > 1:
        multi_pipe_count += 1
    else:
        single_or_none_pipe_count += 1

# Tambahkan kolom ke dataset
test_data = test_data.add_column("truncated_decoded_passage", decoded_passages)
test_data = test_data.add_column("pipe_count", pipe_counts)

# Hitung total dan persentase
total = multi_pipe_count + single_or_none_pipe_count
percent_multi = 100 * multi_pipe_count / total
percent_single_or_none = 100 * single_or_none_pipe_count / total

# Cetak statistik
print(f"Total entries: {total}")
print(f"Pipe > 1: {multi_pipe_count} ({percent_multi:.2f}%)")
print(f"Pipe ≤ 1: {single_or_none_pipe_count} ({percent_single_or_none:.2f}%)")

# # Contoh hasil
# print("\nContoh hasil:")
# for i in range(3):
#     print(f"\nOriginal passage:\n{test_data['passages'][i]}")
#     print(f"\nTruncated-decoded passage:\n{test_data['truncated_decoded_passage'][i]}")
#     print(f"Jumlah karakter '|': {test_data['pipe_count'][i]}")


In [9]:
test_data

Dataset({
    features: ['query', 'passages', 'summary', 'final_summary', 'answer', 'generated_results'],
    num_rows: 565
})

# Generate Answer based on T5 Summary

Sebelum menjalankan generasi jawaban dengan rangkuman hasil fine tuning T5, T5 sudah melakukan generasi rangkuman ketika menjalankan train_summarizer.py dengan command `python train_summarizer.py --do_predict`

In [1]:
from utils import load_model_and_tokenizer
model_name='Qwen/Qwen3-1.7B'
model, tokenizer, config = load_model_and_tokenizer(model_name)

/data/khalid_RAG/RAG-way/rag_way/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 23.42it/s]


In [52]:
from datasets import load_from_disk
dataset = load_from_disk("./generated_data/RECOMP_tuning_with_no_Judul_nTeks")
test_data = dataset["test"]

In [54]:
test_data

Dataset({
    features: ['query', 'passages', 'summary', 'final_summary', 'answer', 'generated_results'],
    num_rows: 565
})

In [19]:
from generate_answer import generate_answer_and_do_scoring

# Muat baris-baris teks dari file
with open("./outputs/-.-models--google-flan-t5-base-2025-05-28_07-17-31-2025-05-28_10-26-57/generated_predictions.txt", "r", encoding="utf-8") as f:
    predictions = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("T5_summary", predictions)

answer_based_on_t5_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="T5_summary", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

Generating responses with summary only: 100%|██████████| 565/565 [12:34<00:00,  1.34s/it]


In [22]:
from datasets import Dataset
fin = Dataset.from_list(answer_based_on_t5_summary)

In [34]:
fin.save_to_disk("./outputs/EM_F1_testing/SCORES_google-flan-t5-base-2025-05-28_07-17-31")

Saving the dataset (1/1 shards): 100%|██████████| 565/565 [00:00<00:00, 69336.47 examples/s]


In [30]:
fin_read = load_from_disk("./outputs/EM_F1_testing/SCORES_google-flan-t5-base-2025-05-28_07-17-31")
fin_read

Dataset({
    features: ['query', 'passages', 'summary', 'label', 'generated_answer', 'em', 'f1'],
    num_rows: 565
})

In [24]:
sum(fin['em'])/len(fin)

0.42123893805309737

In [50]:
sum(substringmatch_xcode_details)

226.0

In [51]:
len(substringmatch_xcode_details)

565

In [25]:
sum(fin['f1'])/len(fin)

0.25522817089095584

# Menguji kinerja jika menggunakan RAG biasa

In [ ]:
from generate_answer import generate_answer_and_do_scoring
answer_based_on_passages = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="passages", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52, 
    max_source_length = 512
)

In [8]:
from datasets import Dataset
normal_RAG = Dataset.from_list(answer_based_on_passages)
print("HASIL RAG BIASA")
print("rerata EM:", sum(normal_RAG['em'])/len(normal_RAG))
print("rerata F1:", sum(normal_RAG['f1'])/len(normal_RAG))

HASIL RAG BIASA
rerata EM: 0.29557522123893804
rerata F1: 0.1659491847867348


__Terbukti bahwa dengan menggunakan RECOMP kinerja RAG menjadi lebih baik, bisa dilihat dari skor EM dan F1 nya yang lebih tinggi__

# Push Model ke Huggingface Hub

In [18]:
t5_model_path = "./models/-google-flan-t5-base-2025-05-28_07-17-31"

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_path)
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_path)

In [ ]:
hub_repo = "khalidrizki/indonesian-T5-RECOMP"
t5_model.push_to_hub(hub_repo)
t5_tokenizer.push_to_hub(hub_repo)